<a href="https://colab.research.google.com/github/GassnerChristoph/soil-lead-project/blob/patch-1/Fig1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Figure 1: Comparing backscatter and transmission geometries

* For 100 ppm Pb in soil
* 10h exposure of the soil in both geometries
* Data is provided in Google Drive

-> Script computes the visualized data and displays the plot itself in the last cell...

In [1]:
!pip install -q gdown pandas numpy matplotlib
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from time import time

Loading the Data from Drive (Folder Fig1: https://drive.google.com/drive/folders/1bFMJsZ8N_UUxOB5lu_1zO6FvoXilRRmm?usp=drive_link)

In [2]:
start_time = time()
print("Reading simulated data...")

id_backscatter = "1XobbLRaWrPsWE1FHDcAFQWGz-CpIreDW"
id_transmission = "1I4-G6H5kvFTSR_BDoRZJxooL8v4CKT43"
gdown.download(id=id_backscatter, output="data_backscatter_realistic_108particles_100ppm.csv", quiet=False)
gdown.download(id=id_transmission, output="data_transmission_108particles_100ppm.csv", quiet=False)

data_b = pd.read_csv("data_backscatter_realistic_108particles_100ppm.csv")
data_t = pd.read_csv("data_transmission_108particles_100ppm.csv")

print(f"Done reading files (took {time() - start_time:.1f} s)")

Reading simulated data...


Downloading...
From: https://drive.google.com/uc?id=1XobbLRaWrPsWE1FHDcAFQWGz-CpIreDW
To: /content/data_backscatter_realistic_108particles_100ppm.csv
100%|██████████| 23.1M/23.1M [00:00<00:00, 172MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1I4-G6H5kvFTSR_BDoRZJxooL8v4CKT43
From (redirected): https://drive.google.com/uc?id=1I4-G6H5kvFTSR_BDoRZJxooL8v4CKT43&confirm=t&uuid=1c13817b-f4fb-4a35-bfe3-760accf8a6fe
To: /content/data_transmission_108particles_100ppm.csv
100%|██████████| 2.16G/2.16G [00:19<00:00, 109MB/s] 


Done reading files (took 85.3 s)


Processing

In [3]:
#   Add energy resolution

FWHM_keV = 0.15  # [keV]
sigma = FWHM_keV / 2.355
rng = np.random.default_rng()

print(f"Adding realistic energy resolution: {FWHM_keV:.2f} keV ...")

# Energie (MeV → keV) + Gaußrauschen
data_b["E"] = data_b["Energy(MeV)"] * 1000.0 + sigma * rng.standard_normal(len(data_b))
data_t["E"] = data_t["Energy(MeV)"] * 1000.0 + sigma * rng.standard_normal(len(data_t))

print("Done adding energy resolution.")

#   Filter events (10 hours, cone geometry)

fluence = 995 * np.pi * 2 * (1 - np.cos(np.pi / 4))  # photons/s in 90° cone
N = 10 * 60 * 60 * fluence                           # photons in 10h

detector_area = 50.0  # mm²
detector_radius = np.sqrt(detector_area / np.pi)

for df in [data_b, data_t]:
    df["r_hit"] = np.sqrt(df["x(mm)"] ** 2 + df["y(mm)"] ** 2)

mask_b = (
    (data_b["EventID"] < N)
    & (data_b["Particle"] == "gamma")
    & (data_b["r_hit"] <= detector_radius)
)
mask_t = (
    (data_t["EventID"] < N)
    & (data_t["Particle"] == "gamma")
    & (data_t["r_hit"] <= detector_radius)
)

data_b = data_b[mask_b]
data_t = data_t[mask_t]

primary_b = data_b[data_b["Type"] == "Primary"]
secondary_b = data_b[data_b["Type"] == "Secondary"]

primary_t = data_t[data_t["Type"] == "Primary"]
secondary_t = data_t[data_t["Type"] == "Secondary"]

#   Histogram conversion

binwidth = 0.03  # [keV]
E = np.arange(0, 75 + binwidth, binwidth)

def hist(data):
    return np.histogram(data["E"], bins=np.append(E, E[-1] + binwidth))[0]

b = {
    "primary": hist(primary_b),
    "secondary": hist(secondary_b),
}
t = {
    "primary": hist(primary_t),
    "secondary": hist(secondary_t),
}

Adding realistic energy resolution: 0.15 keV ...
Done adding energy resolution.


Calculating the SNR Values


*   Calculating the SNR values exact after 10h
*   Also store the exact number of signal and background counts
*   SNR = N_signal/sqrt(N_signal + N_background)


In [4]:
peak = {
    "alpha": {"E": 10.52},
    "beta": {"E": 12.65},
}
peak_window = FWHM_keV

for name in ["alpha", "beta"]:
    mask = (E >= peak[name]["E"] - peak_window) & (E <= peak[name]["E"] + peak_window)
    peak[name]["window"] = mask

def calc_snr(hist_primary, hist_secondary, peak):
    Np = np.sum(hist_primary[peak])
    Ns = np.sum(hist_secondary[peak])
    SNR = Ns / np.sqrt(Ns + Np) if Ns + Np > 0 else 0
    return Np, Ns, SNR

for geom, d in zip(["b", "t"], [b, t]):
    for name in ["alpha", "beta"]:
        Np, Ns, SNR = calc_snr(d["primary"], d["secondary"], peak[name]["window"])
        d[name] = {"primary": Np, "secondary": Ns, "SNR": SNR}

print("Done calculating SNR values")

Plotting the Geometry

In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Geometry-Parameter
sample_thickness_transmission = 2.5  # mm
sample_thickness_backscatter = 25.0  # mm
sample_width = 20.0                  # mm
source_to_surface = 2.0
detector_to_surface = 3.0
detector_height = 0.2
detector_width = 8.0

xLimGeom = [-10, 10]
yLimGeom = [-4, 7]

fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        "Backscatter geometry",
        "Transmission geometry",
        "Backscatter geometry",
        "Transmission geometry",
        "Backscatter geometry (zoom)",
        "Transmission geometry (zoom)",
    ),
    horizontal_spacing=0.08,
    vertical_spacing=0.08
)

# -------------------------------
# BACKSCATTER GEOMETRY
# -------------------------------
sample_y = sample_thickness_transmission - sample_thickness_backscatter

# Probe
fig.add_shape(
    type="rect",
    x0=-sample_width/2,
    x1=sample_width/2,
    y0=sample_y,
    y1=sample_y + sample_thickness_backscatter,
    fillcolor="rgb(230, 204, 179)",  # (0.9, 0.8, 0.7)
    line=dict(color="rgba(0,0,0,0)"),
    row=1, col=1
)

# Source
source_x = -5.5
source_y = sample_y + sample_thickness_backscatter + source_to_surface
fig.add_trace(
    go.Scatter(
        x=[source_x],
        y=[source_y],
        mode="markers",
        marker=dict(size=10, color="rgb(179,179,179)"),
        showlegend=False
    ),
    row=1, col=1
)

# Detector
detector_x = -0.5
detector_y = sample_y + sample_thickness_backscatter + detector_to_surface
fig.add_shape(
    type="rect",
    x0=detector_x,
    x1=detector_x + detector_width,
    y0=detector_y,
    y1=detector_y + detector_height,
    fillcolor="rgb(153,153,153)",
    line=dict(color="black", width=1),
    row=1, col=1
)

# Cone beam
cone_len = 30
angles = np.deg2rad([-5, -95])
cone_x = [
    source_x,
    source_x + cone_len * np.cos(angles[0]),
    source_x + cone_len * np.cos(angles[1]),
    source_x,
]
cone_y = [
    source_y,
    source_y + cone_len * np.sin(angles[0]),
    source_y + cone_len * np.sin(angles[1]),
    source_y,
]
fig.add_trace(
    go.Scatter(
        x=cone_x,
        y=cone_y,
        fill="toself",
        fillcolor="rgba(77,179,255,0.2)",
        line=dict(color="rgba(0,0,0,0)"),
        showlegend=False
    ),
    row=1, col=1
)

fig.update_xaxes(range=xLimGeom, row=1, col=1, showgrid=True, zeroline=False)
fig.update_yaxes(range=yLimGeom, row=1, col=1, scaleanchor="x", scaleratio=1, showgrid=True, zeroline=False)

# -------------------------------
# TRANSMISSION GEOMETRY
# -------------------------------
# Probe
fig.add_shape(
    type="rect",
    x0=-sample_width/2,
    x1=sample_width/2,
    y0=0,
    y1=sample_thickness_transmission,
    fillcolor="rgb(230, 204, 179)",
    line=dict(color="rgba(0,0,0,0)"),
    row=1, col=2
)

# Source
source_x_t = 0
source_y_t = sample_thickness_transmission + source_to_surface
fig.add_trace(
    go.Scatter(
        x=[source_x_t],
        y=[source_y_t],
        mode="markers",
        marker=dict(size=10, color="rgb(179,179,179)"),
        showlegend=False
    ),
    row=1, col=2
)

# Detector
detector_y_t = -detector_to_surface - detector_height
fig.add_shape(
    type="rect",
    x0=source_x_t - detector_width/2,
    x1=source_x_t + detector_width/2,
    y0=detector_y_t,
    y1=detector_y_t + detector_height,
    fillcolor="rgb(153,153,153)",
    line=dict(color="black", width=1),
    row=1, col=2
)

# Kegel
angles_t = np.deg2rad([-45, -135])
cone_x_t = [
    source_x_t,
    source_x_t + cone_len * np.cos(angles_t[0]),
    source_x_t + cone_len * np.cos(angles_t[1]),
    source_x_t,
]
cone_y_t = [
    source_y_t,
    source_y_t + cone_len * np.sin(angles_t[0]),
    source_y_t + cone_len * np.sin(angles_t[1]),
    source_y_t,
]
fig.add_trace(
    go.Scatter(
        x=cone_x_t,
        y=cone_y_t,
        fill="toself",
        fillcolor="rgba(77,179,255,0.2)",
        line=dict(color="rgba(0,0,0,0)"),
        showlegend=False
    ),
    row=1, col=2
)

fig.update_xaxes(range=xLimGeom, row=1, col=2, showgrid=True, zeroline=False)
fig.update_yaxes(range=yLimGeom, row=1, col=2, scaleanchor="x", scaleratio=1, showgrid=True, zeroline=False)

print("Done generating geometry plots")

Done generating geometry plots


Plotting the Spectrums

In [20]:

def add_spectrum(row, col, E, primary, secondary, peak, xlim, ylim, title, data, showlegend=False):
    # 1) Signal
    fig.add_trace(
        go.Scatter(
            x=E,
            y=primary + secondary,
            fill='tozeroy',
            line=dict(color="rgb(201,24,31)"),
            name="Signal",
            showlegend=showlegend,
        ),
        row=row, col=col
    )

    # 2) Background
    fig.add_trace(
        go.Scatter(
            x=E,
            y=primary,
            fill='tozeroy',
            line=dict(color="rgb(0,114,189)"),
            fillcolor="rgb(0,114,189)",  # voll deckend
            name="Background",
            showlegend=showlegend,
        ),
        row=row, col=col
    )

    # Peak-Fenster-Linien wie gehabt
    for name in ["alpha", "beta"]:
        fig.add_vline(
            x=peak[name]["E"] - peak_window,
            line_dash="dash",
            line_color="black",
            row=row, col=col
        )
        fig.add_vline(
            x=peak[name]["E"] + peak_window,
            line_dash="dash",
            line_color="black",
            row=row, col=col
        )

    fig.update_xaxes(range=xlim, title_text="Energy (keV)", row=row, col=col)
    fig.update_yaxes(range=ylim, title_text="Detected counts", row=row, col=col)

    subtitle = (
        f"Lα: Nₛ={int(data['alpha']['secondary'])}, N_b={int(data['alpha']['primary'])} → SNR={int(round(data['alpha']['SNR']))}<br>"
        f"Lβ: Nₛ={int(data['beta']['secondary'])}, N_b={int(data['beta']['primary'])} → SNR={int(round(data['beta']['SNR']))}"
    )
    fig.add_annotation(
        x=0.5,
        y=0.98,
        xref=f"x{2*(row-1)+col} domain",
        yref=f"y{2*(row-1)+col} domain",
        text=subtitle,
        showarrow=False,
        font=dict(size=8),
        align="left"
    )

    # Backscatter full scale
add_spectrum(
    row=2, col=1,
    E=E,
    primary=b["primary"],
    secondary=b["secondary"],
    peak=peak,
    xlim=(5, 60),
    ylim=(0, 400),
    title="Backscatter geometry",
    data=b,
    showlegend=True  # nur hier Legende zeigen
)

# Transmission full scale
add_spectrum(
    row=2, col=2,
    E=E,
    primary=t["primary"],
    secondary=t["secondary"],
    peak=peak,
    xlim=(5, 60),
    ylim=(0, 40000),
    title="Transmission geometry",
    data=t,
    showlegend=False
)

# Backscatter zoom
add_spectrum(
    row=3, col=1,
    E=E,
    primary=b["primary"],
    secondary=b["secondary"],
    peak=peak,
    xlim=(10, 13),
    ylim=(0, 110),
    title="Backscatter geometry (zoom)",
    data=b,
    showlegend=False
)

# Transmission zoom
add_spectrum(
    row=3, col=2,
    E=E,
    primary=t["primary"],
    secondary=t["secondary"],
    peak=peak,
    xlim=(10, 13),
    ylim=(0, 110),
    title="Transmission geometry (zoom)",
    data=t,
    showlegend=False
)

print("Done generating spectrum plots")

Done generating spectrum plots


Display the Plot


In [21]:
fig.update_layout(
    height=900,
    width=900,
    title="Geometry comparison for Am241 (100 ppm, 10 hours)",
    font=dict(size=10),
    margin=dict(t=80, l=40, r=20, b=40)
)

fig.show()